# Road Following Data Editing - Adding xy Coordinates to Photos by Mouseclick

Author: Travis Moore, NNMC STARR Team

Note: This code draws heavily from three sources. First, the NVIDIA data_collection.ipynb notebook shipped with the Jetbot sold by Sparkfun.com. You can find the original code here: https://github.com/NVIDIA-AI-IOT/jetbot

Second, George Gorospe's modified data collection notebook that uses the Jupyter Clickable Widgets library to select a best path by mouseclick rather than scrollbars (https://github.com/jaybdub/jupyter_clickable_image_widget). 

Third, the BalticBot Poland STARR team's fantastic notebook that provides functionality to mark X,Y coordinates on existing images.

In [58]:
# Import libraries
# IPython Libraries for display and widgets
import ipywidgets.widgets as widgets
import traitlets
from IPython.display import display
from ipywidgets import interact, interact_manual
from jupyter_clickable_image_widget import ClickableImageWidget

# Python basic packages for image annotation
import os
import numpy as np
import cv2
import time

count = -1

# Create a new directory to save the modified images
NEW_DIR = 'dataset_test_new'
try:
    os.makedirs(NEW_DIR)
except FileExistsError:
    print('Directory not created because it already exists')

# Create a list of photos from the existing directory
OLD_DIR = 'dataset_test/test'
photo_list = os.listdir(OLD_DIR)
nr_photos = len(photo_list)
print("You have %d photos in this folder." %nr_photos)

# Create a widget to display the current image
image_widget = ClickableImageWidget(value = imv, format='jpeg', width=224, height=224)

# Create a widget to display the snapshot after mouseclick
snapshot_widget = widgets.Image(width=224, height=224)

# Create a save button
button_layout = widgets.Layout(width='128px', height='64px', align_self='center')
save_button = widgets.Button(description='SAVE', button_style='success', layout=button_layout)

# Container for widgets
widget_container = widgets.VBox([widgets.HBox([image_widget])])

# Container for button
button_container = widgets.VBox([widgets.HBox([save_button])])

# Create a placeholder image to identify the end of the dataset
end_img = np.zeros((224,224,3), np.uint8)
font = cv2.FONT_HERSHEY_COMPLEX
cv2.putText(end_img,'END OF DATASET',(60,120), font, 0.4,(255,255,255),1,cv2.LINE_AA)
imv_end = cv2.imencode('.jpg', end_img)[1].tobytes()

# Draw a circle on the image and store the x,y coordinates from mouseclick 
def save_snapshot(_, content, msg):
    global x, y
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # Display new snapshot with Circle marking
        snapshot = np.copy(img)
        snapshot = cv2.circle(snapshot, (x,y), 8, (0,255,0), 3)
        snapshot = cv2.imencode('.jpg', snapshot)[1].tobytes()
        image_widget.value = snapshot
    
# Apply the x and y offset data to the current image's name and save in the new directory
def save_photo():
    file = photo_list[count]

    new_file = 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, file)

    image_path = os.path.join(DATASET_XY_DIR, new_file)
    with open(image_path, 'wb') as f:
        f.write(imv)
    # Move to next image
    next_photo()

# Display the current image in the set
def next_photo():
    global count, file, img, imv, more_images
    count = (count + 1)
    if count < nr_photos:
        file = photo_list[count]
        print(file)
        img = cv2.imread(DIR + '/' + file)
        imv = cv2.imencode('.jpg', img)[1].tobytes()
        image_widget.value = imv
    else:
        image_widget.value = imv_end
        

# Display the first image        
next_photo()

# Use the clickable widget to call the save_snapshot method
image_widget.on_msg(save_snapshot)

# Assign function to save button
save_button.on_click(lambda x: save_photo())

display(widget_container)
display(button_container)

Directory not created because it already exists
You have 3 photos in this folder.
7558d178-7df0-11eb-add8-08beac06b6d8.jpg
